In [45]:
import json, sys, re, torch
import numpy as np
import torch.nn.functional as F
from collections import Counter
sys.path.append("/home/lamdo/keyphrase_informativeness_test/splade/splade")
sys.path.append("/home/lamdo/keyphrase_informativeness_test/src")
sys.path.append("/home/lamdo/keyphrase_informativeness_test/src")
from nounphrase_extractor import CandidateExtractorRegExpNLTK
from nltk.tokenize import sent_tokenize as nltk_sent_tokenize
from transformers import AutoTokenizer, AutoModel
from utils import clean_text
from tqdm import tqdm
from splade.models.transformer_rep import Splade
from functools import lru_cache

In [2]:
CANDEXT = CandidateExtractorRegExpNLTK([1,5])

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device '{DEVICE}'")
SPLADE_MODEL = {}

def init_splade_model(model_name):
    if SPLADE_MODEL.get(model_name) is not None:
        return 
    else:
        print(f"Init splade model {model_name}. This will be done only once")

    if model_name == "custom_trained_pubmedqa+specter":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_pubmedqa+specter/debug/checkpoint/model"

    elif model_name == "custom_trained_msmarco":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_msmarco/debug/checkpoint/model"

    elif model_name == "splade-cocondenser-ensembledistil":
        model_type_or_dir = "naver/splade-cocondenser-ensembledistil"

    elif model_name == "splade-cocondenser-selfdistil":
        model_type_or_dir = "naver/splade-cocondenser-selfdistil"
    elif model_name == "custom_trained_scirepeval_search":
        model_type_or_dir = "/scratch/lamdo/splade_checkpoints/experiments_scirepeval/debug/checkpoint/model"
    elif model_name == "custom_trained_scirepeval_search_v2":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_scirepeval_search_v2/debug/checkpoint/model"
    elif model_name == "custom_trained_scirepeval_highly_influential_citation":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_scirepeval_high_influential_citation/debug/checkpoint/model"
    elif model_name == "custom_trained_scirepeval_highly_influential_citation->search":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_scirepeval_high_influential_citation->search/debug/checkpoint/model"
    elif model_name == "custom_trained_scirepeval_highly_influential_citation+search":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_scirepeval_high_influential_citation+search/debug/checkpoint/model"
    elif model_name == "custom_trained_unarxive_citation_context":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments/debug/checkpoint/model"
    elif model_name == "custom_trained_unarxive_citation_context->scirepeval_search":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_unarxive--scirepeval_search/debug/checkpoint/model"
    elif model_name == "custom_trained_unarxive_citation_context+scirepeval_search":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_unarxive+scirepeval_search/debug/checkpoint/model"
    elif model_name == "custom_trained_unarxiv_citation_context_random_negsampling_only":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_unarxiv_citation_context_random_negsampling_only_2/debug/checkpoint/model"
    elif model_name == "custom_trained_unarxiv_intro_related_work":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_unarxive_intro_related_work/debug/checkpoint/model"
    elif model_name == "custom_trained_unarxive_full_paper":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_unarxive_full_paper/debug/checkpoint/model"
    elif model_name == "custom_trained_unarxive_full_paper_v2":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_unarxive_full_paper_v2/debug/checkpoint/model"
    elif model_name == "custom_trained_unarxive_full_paper->scirepeval_search":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_unarxive_full_paper->scirepeval_search/debug/checkpoint/model"
    elif model_name == "custom_trained_unarxive_full_paper_v2":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_unarxive_full_paper_v2/debug/checkpoint/model"
    elif model_name == "custom_trained_unarxive_full_paper_v2->scirepeval_search_v2":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_unarxive_full_paper_v2->scirepeval_search_v2/debug/checkpoint/model"
    elif model_name == "custom_trained_unarxive_intro_relatedwork_1citationpersentence":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_unarxive_intro_relatedwork_1citationpersentence/debug/checkpoint/model"
    elif model_name == "custom_trained_unarxive_intro_relatedwork_1citationpersentence->scirepeval_search":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_unarxive_intro_relatedwork_1citationpersentence->scirepeval_search/debug/checkpoint/model"
    elif model_name == "custom_trained_unarxive_full_paper_1citationpersentence":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_unarxive_full_paper_1citationpersentence/debug/checkpoint/model"
    elif model_name == "custom_trained_unarxive_full_paper_1citationpersentence->scirepeval_search":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_unarxive_full_paper_1citationpersentence->scirepeval_search/debug/checkpoint/model"
    elif model_name == "custom_trained_splade_doc_unarxive_full_paper_1citationpersentence":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_splade_doc_unarxive_full_paper_1citationpersentence/debug/checkpoint/model"
    elif model_name == "custom_trained_splade_doc_unarxive_full_paper_1citationpersentence->scirepeval_search":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_splade_doc_unarxive_full_paper_1citationpersentence->scirepeval_search/debug/checkpoint/model"
    elif model_name == "custom_trained_unarxive_intro_relatedwork_1citationpersentence+scirepeval_search":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_unarxive_intro_relatedwork_1citationpersentence+scirepeval_search/debug/checkpoint/model"
    elif model_name == "custom_trained_scirepeval_search->unarxive_intro_relatedwork_1citationpersentence":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_scirepeval_search->unarxive_intro_relatedwork_1citationpersentence/debug/checkpoint/model"
    elif model_name == "custom_trained_unarxive_intro_relatedwork_1citationpersentence+scirepeval_search_v2":
        model_type_or_dir = "/home/lamdo/keyphrase_informativeness_test/splade/experiments_unarxive_intro_relatedwork_1citationpersentence+scirepeval_search_v2/debug/checkpoint/model"
    elif model_name == "custom_trained_combined_references":
        model_type_or_dir = "/scratch/lamdo/splade_checkpoints/experiments_combined_references/debug/checkpoint/model"
    elif model_name == "custom_trained_combined_references_v2":
        model_type_or_dir = "/scratch/lamdo/splade_checkpoints/experiments_combined_references_v2/debug/checkpoint/model"
    else: raise NotImplementedError

    model = Splade(model_type_or_dir, agg="max")
    model = model.to(DEVICE)
    model.eval()

    tokenizer = AutoTokenizer.from_pretrained(model_type_or_dir)
    reverse_voc = {v: k for k, v in tokenizer.vocab.items()}

    SPLADE_MODEL[model_name] = {
        "model": model,
        "tokenizer": tokenizer,
        "reverse_voc": reverse_voc
    }

Using device 'cuda'


In [46]:
def get_tokens_scores_of_doc(doc, model_name):
    try:
        # now compute the document representation
        with torch.no_grad():
            doc_rep = SPLADE_MODEL[model_name]["model"](d_kwargs=SPLADE_MODEL[model_name]["tokenizer"](doc, return_tensors="pt", max_length = 512, truncation = True).to(DEVICE))["d_rep"].squeeze().cpu()  # (sparse) doc rep in voc space, shape (30522,)

        # get the number of non-zero dimensions in the rep:
        col = torch.nonzero(doc_rep).squeeze().cpu().tolist()
        # print("number of actual dimensions: ", len(col))

        # now let's inspect the bow representation:
        weights = doc_rep[col].cpu().tolist()
        d = {k: v for k, v in zip(col, weights)}
        sorted_d = {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}
        bow_rep = []
        for k, v in sorted_d.items():
            bow_rep.append((SPLADE_MODEL[model_name]["reverse_voc"][k], round(v, 2)))
        # print("SPLADE BOW rep:\n", bow_rep)

        return Counter({line[0]: line[1] for line in bow_rep})
    except Exception as e:
        print(e)
        return Counter()
    

def score_candidates_by_positions(candidates, doc):
    res = Counter()
    for cand in candidates:
        try:
            temp = doc.index(cand)
            position = len([item for item in doc[:temp].split(" ") if item]) + 1
            position_score = (position + 1) / position
        except ValueError:
            position_score = 0
        res[cand] = position_score
    return res

def score_candidates(candidates, 
                     tokens_scores, 
                     model_name, 
                     length_penalty = 0,
                     candidates_positions_scores = {}):
    # length penalization < 0 means returning longer sequence
    tokenized_candidates = [SPLADE_MODEL[model_name]["tokenizer"].convert_ids_to_tokens(SPLADE_MODEL[model_name]["tokenizer"](cand)["input_ids"][1:-1]) for cand in candidates]

    candidates_scores = [np.sum([tokens_scores[tok] for tok in tokenized_cand]) / (len(tokenized_cand) - length_penalty) for tokenized_cand in tokenized_candidates]

    if candidates_positions_scores:
        candidates_scores = [score * candidates_positions_scores[candidates[i]] for i, score in enumerate(candidates_scores)]
    assert len(candidates) == len(candidates_scores)
    return [(cand, score) for cand, score in zip(candidates, candidates_scores)]


@lru_cache(maxsize = 10000)
def keyphrase_extraction(doc_to_get_token_score,
                         doc_to_get_candidates, 
                         top_k = 10, 
                         model_name = "splade-cocondenser-ensembledistil",
                         apply_position_penalty = False,
                         length_penalty = 0):
    init_splade_model(model_name)

    lower_doc_to_get_candidates = doc_to_get_candidates.lower()
    lower_doc_to_get_token_score = doc_to_get_token_score.lower()

    tokens_scores = get_tokens_scores_of_doc(doc_to_get_token_score, model_name = model_name)
    candidates = CANDEXT(lower_doc_to_get_candidates)

    if apply_position_penalty:
        candidates_positions_scores = score_candidates_by_positions(candidates, lower_doc_to_get_candidates)
    else:
        candidates_positions_scores = []

    # print(candidates[0], tokens_scores[candidates[0]], candidates_positions_scores[candidates[0]])
    scores = score_candidates(candidates, 
                              tokens_scores, 
                              model_name = model_name, 
                              length_penalty=length_penalty,
                              candidates_positions_scores = candidates_positions_scores)


    return list(sorted(scores, key = lambda x: -x[1]))[:top_k]

In [23]:
init_splade_model("custom_trained_scirepeval_search_v2")

In [6]:
get_tokens_scores_of_doc("this is a test sentence", "custom_trained_scirepeval_search_v2")

Counter({'test': 2.36,
         'sentence': 2.22,
         'this': 1.41,
         'trial': 1.21,
         'here': 1.2,
         'is': 1.18,
         '.': 1.15,
         ',': 0.7,
         'testing': 0.67,
         'exam': 0.63,
         'a': 0.52,
         'now': 0.51,
         'be': 0.42,
         'the': 0.38,
         'tests': 0.37,
         ':': 0.37,
         'by': 0.29,
         'term': 0.27,
         'to': 0.26,
         'he': 0.22,
         'in': 0.2,
         '-': 0.2,
         'and': 0.2,
         '"': 0.14,
         'she': 0.07,
         'him': 0.06})

In [7]:
with open("/scratch/lamdo/unArxive/triplets_full_paper_1citationpersentence.json") as f:
    triplets_data = json.load(f)

In [15]:
triplets_data[200]

['In  and in the present models the\nbreaking of the extra U(1) surviving\nmassless the Green-Schwarz mechanism proceeds via tachyonic excitations in the sector\naccommodating the right handed neutrino.',
 'The Standard Model from Stable Intersecting Brane World Orbifolds. We analyze the perturbative stability of non-supersymmetric intersecting\nbrane world models on tori. Besides the dilaton tadpole, a dynamical\ninstability in the complex structure moduli space occurs at string disc level,\nwhich drives the background geometry to a degenerate limit. We show that in\ncertain orbifold models this latter instability is absent as the relevant\nmoduli are frozen. We construct explicit examples of such orbifold intersecting\nbrane world models and discuss the phenomenological implications of a three\ngeneration Standard Model which descends naturally from an SU(5) GUT theory. It\nturns out that various phenomenological issues require the string scale to be\nat least of the order of the GUT

In [13]:
len('The interested reader is referred to  for the details.'.split(" "))

10

In [31]:
anchor, pos, neg = triplets_data[200]
keyphrase_extraction(doc_to_get_token_score=f"{pos}", 
                                      doc_to_get_candidates=anchor, 
                                      top_k = 10, 
                                      model_name="custom_trained_scirepeval_search_v2")[:2]

[('green-schwarz mechanism proceeds', 0.266), ('present models', 0.0)]

In [47]:
new_triplets_data = []
for triplet in tqdm(triplets_data[:]):
    anchor, pos, neg = triplet
    # if len(anchor.split(" ")) < 20: continue

    title_pos = pos.split(". ")[0]
    keyphrases = keyphrase_extraction(doc_to_get_token_score=title_pos, 
                                      doc_to_get_candidates=anchor, 
                                      top_k = 10, 
                                      model_name="custom_trained_scirepeval_search_v2")[:2]
    keyphrases = [item[0] for item in keyphrases if item[1]]

    for keyphrase in keyphrases:
        new_triplet = [keyphrase, pos, neg]

        new_triplets_data.append(new_triplet)

  0%|          | 3011/3900205 [00:08<2:53:40, 373.99it/s]


KeyboardInterrupt: 

In [43]:
new_triplets_data[:]

[]

In [1]:
import json


with open("/scratch/lamdo/unArxive/triplets_full_paper_1citationpersentence_mined_phrases.json") as f:
    data = json.load(f)

In [2]:
data

[['quantum',
  'Quantum generalized cohomology. We construct a ring structure on complex cobordism tensored with the\nrationals, which is related to the usual ring structure as quantum cohomology\nis related to ordinary cohomology. The resulting object defines a generalized\ntwo- dimensional topological field theory taking values in a category of\nspectra.',
  "Quantum K-theory on flag manifolds, finite-difference Toda lattices and quantum groups. We conjecture that appropriate K-theoretic Gromov-Witten invariants of\ncomplex flag manifolds G/B are governed by finite-difference versions of Toda\nsystems constructed in terms of the Langlands-dual quantized universal\nenveloping algebras U_q(g'). The conjecture is proved in the case of classical\nflag manifolds of the series A. The proof is based on a refinement of the\nfamous Atiyah-Hirzebruch argument for rigidity of arithmetical genus applied to\nhyperquot-scheme compactifications of spaces of rational curves in the flag\nmanifolds."]